# Day 18
## Part 1
Represent the snail numbers as a list of depth/value pairs. A probably more robust representation would involve a binary tree that tracks leaves to the left and right but this is easier, if a bit hairy in terms of list mutability; the data needs to be copied otherwise it gets changed in place. 

In [1]:
from collections import namedtuple
import math

SnailLeaf = namedtuple('SnailLeaf', 'depth value')


def parse_line(line):
    snailnumber = []
    depth = 0
    for c in line.strip():
        if c == '[': 
            depth += 1
        elif c == ']':
            depth -= 1
        elif c != ',':
            snailnumber.append(SnailLeaf(depth, int(c)))
    return snailnumber
  

def parse_data(s):
    snailnumbers = []
    for line in s.strip().splitlines():
        snailnumbers.append(parse_line(line))
    return snailnumbers


def reduce(sn):
    while True:
        still_exploding = True
        while still_exploding:
            still_exploding = False
            for i, (d, v) in enumerate(sn):
                if d == 5:
                    still_exploding = True
                    if i > 0:
                        sn[i - 1] = SnailLeaf(sn[i - 1].depth, sn[i - 1].value + sn[i].value)
                    if i + 2 < len(sn):
                        sn[i + 2] = SnailLeaf(sn[i + 2].depth, sn[i + 1].value + sn[i + 2].value)
                    sn[i] = SnailLeaf(4, 0)
                    del sn[i + 1]
        split = False
        for i, (d, v) in enumerate(sn):
            if v > 9:
                split = True
                old = sn[i]
                sn[i] = SnailLeaf(old.depth + 1, math.floor(old.value / 2))
                new_leaf = [SnailLeaf(old.depth + 1, math.ceil(old.value / 2))]
                sn = sn[:i + 1] + new_leaf + sn[i + 1:]
                break
        if not split:
            return sn
        
        
reduce(parse_line('[[[[[9,8],1],2],3],4]'))

[SnailLeaf(depth=4, value=0),
 SnailLeaf(depth=4, value=9),
 SnailLeaf(depth=3, value=2),
 SnailLeaf(depth=2, value=3),
 SnailLeaf(depth=1, value=4)]

In [2]:
reduce(parse_line('[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]'))

[SnailLeaf(depth=2, value=3),
 SnailLeaf(depth=3, value=2),
 SnailLeaf(depth=4, value=8),
 SnailLeaf(depth=4, value=0),
 SnailLeaf(depth=2, value=9),
 SnailLeaf(depth=3, value=5),
 SnailLeaf(depth=4, value=7),
 SnailLeaf(depth=4, value=0)]

In [3]:
def add(sn1, sn2):
    sn = [SnailLeaf(d + 1, v) for d, v in sn1 + sn2]
    return reduce(sn)

add(parse_line('[[[[4,3],4],4],[7,[[8,4],9]]]'), parse_line('[1,1]'))

[SnailLeaf(depth=4, value=0),
 SnailLeaf(depth=4, value=7),
 SnailLeaf(depth=3, value=4),
 SnailLeaf(depth=4, value=7),
 SnailLeaf(depth=4, value=8),
 SnailLeaf(depth=4, value=6),
 SnailLeaf(depth=4, value=0),
 SnailLeaf(depth=2, value=8),
 SnailLeaf(depth=2, value=1)]

In [4]:
def magnitude(sn):
    while len(sn) > 1:
        max_depth = max(l.depth for l in sn)
        for i, (d, v) in enumerate(sn):
            if d == max_depth:
                sn[i] = SnailLeaf(d - 1, v * 3 + sn[i + 1].value * 2)
                del sn[i + 1]
    return sn[0].value

magnitude(parse_line('[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]'))

3488

In [5]:
import functools

def part_1(data):
    return magnitude(functools.reduce(add, (x.copy() for x in data)))

In [6]:
test_data = parse_data('''
[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]
''')

assert part_1(test_data)== 4140

In [7]:
data = parse_data(open('input', 'r').read())
part_1(data)

3892

## Part 2

In [8]:
import itertools

def part_2(data):
    return max(magnitude(add(x.copy(), y.copy())) for x, y in itertools.permutations(data, 2))    

In [9]:
assert part_2(test_data) == 3993

In [10]:
part_2(data)

4909